* 하나의 검색 키워드에 대해서만 자동화(어러 종류의 주제를 한번에 크롤링 할 수 있게)
* 한개만 있는 사진만 추출

# import module

In [1]:
import pandas as pd
import numpy as np
import time
import re
import os
import json
from random import random
from selenium.webdriver import Chrome
from urllib.request import urlretrieve
from uuid import uuid4
from io import BytesIO

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request

In [2]:
import warnings
warnings.filterwarnings(action='ignore') 

# 창 띄우기

In [3]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# def set_chrome_driver():
#     chrome_options = webdriver.ChromeOptions()
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
#     return driver

In [4]:
def open_browser():
    browser = Chrome("chromedriver.exe") # 입력
    browser.get("https://www.instagram.com/accounts/login/?hl=ko&source=auth_switcher")
    return browser

# Login

In [5]:
def login(browser):
    id="dg1dgu" # 입력
    pwd="DGU1dgu1" # 입력
    input_id=browser.find_element(By.XPATH,'//*[@id="loginForm"]/div/div[1]/div/label/input') #아이디 입럭 객체 선택
    input_pwd=browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[2]/div/label/input') #패스워드 입력 객체 선택
    input_id.send_keys(id)
    input_pwd.send_keys(pwd)
    time.sleep(1)
    
    browser.find_element(By.XPATH,'//*[@id="loginForm"]/div/div[3]/button/div').click() #로그인 버튼 클릭
    time.sleep(5)
    
    browser.find_element(By.XPATH,'//*[@id="react-root"]/section/main/div/div/div/div/button').click() #로그인정보 저장 나중에하기 버튼
    

# 검색키워드 검색

In [6]:
def search_keyword(browser, keyword):
    url = f"https://www.instagram.com/explore/tags/{keyword}/"
    browser.get(url)

# 첫번째 게시글 오픈

In [7]:
def open_first_post(browser):
    first_post = browser.find_elements_by_class_name("_9AhH0")[0]
    print(type(first_post))
    print(first_post)
    browser.execute_script("arguments[0].click();",first_post)

# 좋아요 출력(넘기면서 반복)

In [8]:
def get_likes(browser):
    try:
        like = browser.find_element_by_xpath('/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span')
        like = like.text.replace(',', '')
        likes = int(like)
        return likes
    except Exception as ex:
        print("get_likes 에러발생", ex)
        return 0

# 이미지 URL, 파일명 출력(넘기면서 반복)

* https://mizykk.tistory.com/55
* openCV를 이용해서 이미지 중복을 처리할 수 있을 듯.

In [9]:
'''
# 앞-현재-뒤 이렇게 컨텐츠가 넘어옴
img = browser.find_element_by_class_name("RnEpo")
img = img.find_elements_by_class_name('KL4Bh') # 동영상이면 
img_url = img[0].find_element_by_tag_name('img').get_attribute("src") #동영상이면 list index out of range 에러뜸
img_url
'''

'\n# 앞-현재-뒤 이렇게 컨텐츠가 넘어옴\nimg = browser.find_element_by_class_name("RnEpo")\nimg = img.find_elements_by_class_name(\'KL4Bh\') # 동영상이면 \nimg_url = img[0].find_element_by_tag_name(\'img\').get_attribute("src") #동영상이면 list index out of range 에러뜸\nimg_url\n'

In [10]:
def get_image_url(browser, keyword):
    try:
        #img = browser.find_elements_by_class_name("KL4Bh") #이렇게 되면 마지막 놈이 뜸. 동영상은 처리 못함.
        img = browser.find_element_by_class_name("RnEpo")
        img = img.find_elements_by_class_name('KL4Bh') # 동영상이면 안뜸.
        #img_url = img[-1].find_element_by_css_selector("article img").get_attribute("src")
        img_url = img[0].find_element_by_tag_name('img').get_attribute("src")

        uuid =uuid4() #유일한 이름 할당. 중복
        #urlretrieve(img_url, f"data/{keyword}/{uuid}.jpg") #이건 다운로드
        img_name = f"{uuid}.jpg"
        return img_url, img_name
    
    except Exception as ex:
        #에러가 발생하면 img_name이 "NaN"으로 출력됨.
        print("get_image 에러발생", ex)
        return "NaN", "NaN"

# 이미지 저장(넘기면서 반복)

In [11]:
def save_image(browser, img_url, keyword, img_name):
    if img_url == 'NaN':
        return 'NaN'
    else:
        save_path = f"image/{keyword}/{img_name}"
        urlretrieve(img_url, save_path)
        print(f"{save_path}에 저장완료")
        return save_path

# 해시태그 출력(NaN 확인 필요) (넘기면서 반복)

In [12]:
def get_hashtag(browser):
    components = browser.find_elements_by_class_name("xil3i")
    #components = browser.find_elements(By.CSS_SELECTOR, "xil3i") #안되는건가?
    if len(components) != 0:
        
        try:
            all_hashtags = [component.text for component in components]
            if len(all_hashtags) > 0: #해시태그가 없으면 빈 리스트 출력
                return list(set(all_hashtags))
            else:
                return []
        
        except Exception as ex:
            print("get_hashtag 에러발생", ex)
            return []

# 이미지 여러장인지 확인(넘기면서 반복)

In [13]:
def check_one_img(browser):
    try:
        browser.find_element_by_class_name('_6CZji')
        return False
    except:
        return True

# 데이터프레임으로 출력&이미지 저장(넘기면서 반복)

| 검색키워드 | 이미지파일명 | 좋아요 | 해시태그 |
| --- | --- | --- | --- |
| 음식스타그램 | asdiim23mmfs.jpg | 좋아요 | #tag1 #tag2 ... |

이런식으로 하면 될라나

In [14]:
def make_df(browser, keyword):
    try:
        # 현재페이지의 #
        #1. 좋아요 가져오기
        likes = get_likes(browser)
        #2. 이미지 파일명 가져오기
        img_url, img_name = get_image_url(browser, keyword)
        
        #3. 이미지 저장하기
        file_path = save_image(browser, img_url, keyword, img_name)
        #4. 해시태그 가져오기
        hashtags = get_hashtag(browser)
        #5. 이미지 1개인지 체크
        one_img = check_one_img(browser)
        
        #6. 한 행의 데이터프레임 생성
        df = pd.DataFrame([[keyword, img_name, file_path, likes, hashtags, one_img]],
                         columns=['검색키워드', '이미지파일명', '파일경로', '좋아요', '해시태그', '이미지한개'])
    
    except Exception as ex:
        print("make_df 에러발생", ex)
        df = pd.DataFrame([[keyword, img_name, file_path, likes, hashtags, one_img]],
                         columns=['검색키워드', '이미지파일명', '파일경로', '좋아요', '해시태그', '이미지한개'])
        
    return df

# 페이지 넘기기

In [15]:
def next_page(browser):
    next_buttons = browser.find_elements_by_xpath('/html/body/div[6]/div[2]/div/div')
    if len(next_buttons) == 2:
        next_button = next_buttons[1]
    else:
        next_button = next_buttons[0]
    next_button.click()

# 디렉토리 생성

In [16]:
def make_dir(keyword):
    try:
        os.makedirs(f"image/{keyword}")
    except OSError:
        print(f'image/{keyword} 경로가 존재합니다.')
        pass

# 자동실행(처음)

In [17]:
def auto_run(keyword, num_post):
    #1. 디렉토리 생성
    make_dir(keyword)
    
    #2. 브라우저 열고 로그인하기
    browser = open_browser()
    time.sleep(2)
    login(browser)
    time.sleep(3)
    
    #3. 검색키워드 입력 후 페이지 전환
    search_keyword(browser, keyword)
    time.sleep(10) #첫 검색은 상당히 오래걸릴 수 있음.
    
    #4. 첫번째 게시글 열기
    open_first_post(browser)
    time.sleep(5 + (random() * 5))
    
    #### 여기서 반복 ####
    total_df_LS = []
    for i in range(num_post):
        #5-1. 데이터프레임으로 출력 및 이미지 저장
        one_row_hashtag_df = make_df(browser, keyword)
        #5-2. 리스트에 저장
        total_df_LS.append(one_row_hashtag_df)
        #5-3. 다음페이지로 이동
        next_page(browser)
        time.sleep(3 + (random() * 5))
        if (i+1)%10 == 0:
            print(f'{i+1}번째 까지 게시글이 스크랩 완료되었습니다.')
            
    #6. 하나의 데이터프레임으로 병합
    total_df = pd.concat(total_df_LS, axis=0)
    total_df.reset_index(inplace=True, drop=True)
    
    return total_df, browser

In [18]:
'''total_df, browser = auto_run('음식스타그램', 5)'''

"total_df, browser = auto_run('음식스타그램', 5)"

In [19]:
'''total_df'''

'total_df'

# 데이터프레임 저장

## 데이터 프레임 디렉토리 생성

In [20]:
def make_dir2(keyword):
    try:
        os.makedirs(f"data/{keyword}")
    except OSError:
        print(f'data/{keyword} 경로가 존재합니다.')
        pass

## 데이터 프레임 저장

In [21]:
def save_df(keyword, df, file_name):
    make_dir2(keyword)
    
    df.to_pickle(f"data/{keyword}/{file_name}.pkl")
    df.to_csv(f"data/{keyword}/{file_name}.csv", encoding='utf-8')
    print(f"data/{keyword}/{file_name}.csv 저장완료")

In [22]:
'''save_df('음식스타그램', total_df, '음식스타그램1')'''

"save_df('음식스타그램', total_df, '음식스타그램1')"

# 실험용 코드(실행X)

In [23]:
'''make_dir('음식스타그램')

#2. 브라우저 열고 로그인하기
browser = open_browser()
time.sleep(2)
login(browser)
time.sleep(3)

#3. 검색키워드 입력 후 페이지 전환
search_keyword(browser, '음식스타그램')
time.sleep(10) #첫 검색은 상당히 오래걸릴 수 있음.

#4. 첫번째 게시글 열기
open_first_post(browser)
time.sleep(5 + (random() * 5))'''

"make_dir('음식스타그램')\n\n#2. 브라우저 열고 로그인하기\nbrowser = open_browser()\ntime.sleep(2)\nlogin(browser)\ntime.sleep(3)\n\n#3. 검색키워드 입력 후 페이지 전환\nsearch_keyword(browser, '음식스타그램')\ntime.sleep(10) #첫 검색은 상당히 오래걸릴 수 있음.\n\n#4. 첫번째 게시글 열기\nopen_first_post(browser)\ntime.sleep(5 + (random() * 5))"

# 분할 실행(한번에 n개 가져오기, k번 실행)

In [24]:
def auto_run_repeat(keyword, num_post, num_repeat):
    #1. 디렉토리 생성
    make_dir(keyword)
    
    #2. 브라우저 열고 로그인하기
    browser = open_browser()
    time.sleep(2)
    login(browser)
    time.sleep(3)
    
    #3. 검색키워드 입력 후 페이지 전환
    search_keyword(browser, keyword)
    time.sleep(10) #첫 검색은 상당히 오래걸릴 수 있음.
    
    #4. 첫번째 게시글 열기
    open_first_post(browser)
    time.sleep(5 + (random() * 5))
    
    #### 여기서 반복 ####
    for n in range(num_repeat):
        print(f'======={n+1}========')
        total_df_LS = []
        for i in range(num_post):
            #5-1. 데이터프레임으로 출력 및 이미지 저장
            one_row_hashtag_df = make_df(browser, keyword)
            #5-2. 리스트에 저장
            total_df_LS.append(one_row_hashtag_df)
            #5-3. 다음페이지로 이동
            next_page(browser)
            time.sleep(3 + (random() * 5))
            if (i+1)%10 == 0:
                print(f'{i+1}번째 까지 게시글이 스크랩 완료되었습니다.')

        #6. 하나의 데이터프레임으로 병합
        total_df = pd.concat(total_df_LS, axis=0)
        total_df.reset_index(inplace=True, drop=True)

        #7. 데이터프레임 저장
        save_df(keyword, total_df, f'{keyword}{n+1}')
    browser.quit()
    time.sleep(2)
    

# 여기서 크롤러 실행하시오!!!

In [26]:
# (검색할 키워드, 1회 실행시 가져올 게시글 수, 반복 횟수)
# auto_run_repeat('맛스타그램', num_post=100, num_repeat=2)
# auto_run_repeat('음식스타그램', num_post=100, num_repeat=2)

# auto_run_repeat('육아', num_post=100, num_repeat=2)
# auto_run_repeat('아기스타그램', num_post=100, num_repeat=2)

# auto_run_repeat('펫스타그램', num_post=100, num_repeat=2)
# auto_run_repeat('반려동물', num_post=100, num_repeat=2)

auto_run_repeat('카페투어', num_post=100, num_repeat=2)
auto_run_repeat('카페탐방', num_post=100, num_repeat=2)

auto_run_repeat('얼스타그램', num_post=100, num_repeat=2)
auto_run_repeat('전신스타그램', num_post=100, num_repeat=2)

auto_run_repeat('북스타그램', num_post=100, num_repeat=2)
auto_run_repeat('독서스타그램', num_post=100, num_repeat=2)

auto_run_repeat('여행일기', num_post=100, num_repeat=2)
auto_run_repeat('여행추억', num_post=100, num_repeat=2)

auto_run_repeat('벌크업', num_post=100, num_repeat=2)
auto_run_repeat('운스타그램', num_post=100, num_repeat=2)

image/카페투어 경로가 존재합니다.
<class 'selenium.webdriver.remote.webelement.WebElement'>
<selenium.webdriver.remote.webelement.WebElement (session="bd549746e6c14026d8f0a3897170df78", element="b0efd4c9-f498-4ed5-85f6-c4911896d468")>
=======1========
image/카페투어/3f6bc85f-4f7b-40a4-b806-2db4a7e459cf.jpg에 저장완료
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/d

get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/d

get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/d

get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/d

get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/d

...

get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/d

get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/d

get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/d

get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 list index out of range
get_likes 에러발생 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span"}
  (Session info: chrome=100.0.4896.127)

get_image 에러발생 Message: no such element: Unable to locate element: {"method":"css selector","selector":".RnEpo"}
  (Session info: chrome=100.0.4896.127)



IndexError: list index out of range